<a href="https://colab.research.google.com/github/ApurvaMayank-iitb/24D0894_IE643_NIFTI/blob/main/Interface_AnoGAN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import files
uploaded = files.upload()  # This will prompt you to upload both .pth files


Saving Discriminator_trained.pth to Discriminator_trained.pth
Saving Generator_trained.pth to Generator_trained.pth


In [3]:
import torch
import torch.nn as nn

# Define device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Generator model definition
class Generator(nn.Module):
    def __init__(self, input_dim=25, output_channels=1):
        super(Generator, self).__init__()

        self.net = nn.Sequential(
            nn.ConvTranspose2d(input_dim, 128, kernel_size=4, stride=1, padding=0),  # 1x1 -> 4x4
            nn.BatchNorm2d(128),
            nn.ReLU(True),

            nn.ConvTranspose2d(128, 64, kernel_size=4, stride=2, padding=1),  # 4x4 -> 8x8
            nn.BatchNorm2d(64),
            nn.ReLU(True),

            nn.ConvTranspose2d(64, 32, kernel_size=4, stride=2, padding=1),  # 8x8 -> 16x16
            nn.BatchNorm2d(32),
            nn.ReLU(True),

            nn.ConvTranspose2d(32, 16, kernel_size=4, stride=2, padding=1),  # 16x16 -> 32x32
            nn.BatchNorm2d(16),
            nn.ReLU(True),

            nn.ConvTranspose2d(16, output_channels, kernel_size=8, stride=8, padding=0),  # 32x32 -> 256x256
            nn.Tanh()
        )

    def forward(self, x):
        x = x.view(x.size(0), x.size(1), 1, 1)  # Reshape to start with a 1x1 input
        return self.net(x)

# Discriminator model definition
class Discriminator(nn.Module):
    def __init__(self, input_channels=1):
        super(Discriminator, self).__init__()

        self.net = nn.Sequential(
            nn.Conv2d(input_channels, 16, kernel_size=4, stride=2, padding=1),  # 256x256 -> 128x128
            nn.LeakyReLU(0.2, inplace=True),

            nn.Conv2d(16, 32, kernel_size=4, stride=2, padding=1),  # 128x128 -> 64x64
            nn.BatchNorm2d(32),
            nn.LeakyReLU(0.2, inplace=True),

            nn.Conv2d(32, 64, kernel_size=4, stride=2, padding=1),  # 64x64 -> 32x32
            nn.BatchNorm2d(64),
            nn.LeakyReLU(0.2, inplace=True),

            nn.Conv2d(64, 128, kernel_size=4, stride=2, padding=1),  # 32x32 -> 16x16
            nn.BatchNorm2d(128),
            nn.LeakyReLU(0.2, inplace=True),

            nn.Conv2d(128, 1, kernel_size=4, stride=1, padding=0),  # 16x16 -> 13x13
            nn.Sigmoid()  # Probability of real or fake
        )

    def forward(self, x):
        return self.net(x).view(-1, 1).squeeze(1)  # Flatten output

# Initialize models
latent_dim = 25
G = Generator(input_dim=latent_dim, output_channels=1).to(device)
D = Discriminator(input_channels=1).to(device)


In [4]:
# Load the trained model weights
G.load_state_dict(torch.load('Generator_trained.pth', map_location=device))
D.load_state_dict(torch.load('Discriminator_trained.pth', map_location=device))

# Set the models to evaluation mode
G.eval()
D.eval()


<ipython-input-4-6538e0a0f0a1>:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  G.load_state_dict(torch.load('Generator_trained.pth', map_location=device))
<ipython-input-4-

Discriminator(
  (net): Sequential(
    (0): Conv2d(1, 16, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
    (1): LeakyReLU(negative_slope=0.2, inplace=True)
    (2): Conv2d(16, 32, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
    (3): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (4): LeakyReLU(negative_slope=0.2, inplace=True)
    (5): Conv2d(32, 64, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
    (6): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (7): LeakyReLU(negative_slope=0.2, inplace=True)
    (8): Conv2d(64, 128, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
    (9): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (10): LeakyReLU(negative_slope=0.2, inplace=True)
    (11): Conv2d(128, 1, kernel_size=(4, 4), stride=(1, 1))
    (12): Sigmoid()
  )
)

In [5]:
import gradio as gr
import nibabel as nib
import numpy as np
import torch
import cv2
import matplotlib.pyplot as plt

# Device setup
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Preprocessing and Averaging Function
def load_and_average_modalities(modalities, slice_axis=2, target_size=(256, 256)):
    processed_slices = []
    for file_path in modalities:
        # Load .nii file
        img = nib.load(file_path)
        data = img.get_fdata()

        # Extract the slice with the most non-zero pixels
        slices = [data.take(i, axis=slice_axis) for i in range(data.shape[slice_axis])]
        max_index = np.argmax([np.count_nonzero(slice_) for slice_ in slices])
        best_slice = slices[max_index]

        # Resize and normalize
        best_slice_resized = cv2.resize(best_slice, target_size)
        best_slice_normalized = best_slice_resized / np.max(best_slice_resized) if np.max(best_slice_resized) > 0 else best_slice_resized
        processed_slices.append(best_slice_normalized)

    # Average across modalities
    averaged_slice = np.mean(processed_slices, axis=0)
    return averaged_slice

# Main Function to Run the Anomaly Detection
def detect_anomaly(*modalities):
    # Step 1: Load and preprocess modalities
    averaged_slice = load_and_average_modalities(modalities)
    validation_tensor = torch.tensor(averaged_slice, dtype=torch.float32).unsqueeze(0).unsqueeze(0).to(device)

    # Step 2: Generate fake image and calculate reconstruction error
    noise = torch.randn((1, latent_dim), device=device)
    with torch.no_grad():
        generated_img = G(noise).squeeze().cpu().numpy()

    real_img_np = validation_tensor.squeeze().cpu().numpy()
    reconstruction_error = np.abs(real_img_np - generated_img)

    # Step 3: Thresholding for anomaly mask
    mean_error = np.mean(reconstruction_error)
    std_error = np.std(reconstruction_error)
    threshold = mean_error + 1.5 * std_error
    anomaly_mask = reconstruction_error > threshold
    anomaly_ratio = np.sum(anomaly_mask) / anomaly_mask.size
    result_text = "Tumor Found!" if anomaly_ratio > 0.04 else "No Tumor Found."

    # Step 4: Plot results
    fig, axes = plt.subplots(1, 4, figsize=(16, 4))
    axes[0].imshow(real_img_np, cmap='gray')
    axes[0].set_title("Original Averaged Image")
    axes[1].imshow(generated_img, cmap='gray')
    axes[1].set_title("Generated Image")
    axes[2].imshow(reconstruction_error, cmap='hot')
    axes[2].set_title("Reconstruction Error")
    axes[3].imshow(anomaly_mask, cmap='gray')
    axes[3].set_title("Anomaly Mask")

    plt.tight_layout()
    plt.close(fig)
    return result_text, fig

# Define Gradio Interface
iface = gr.Interface(
    fn=detect_anomaly,
    inputs=[
        gr.inputs.File(label="Upload T1CE Modality (.nii)", type="file"),
        gr.inputs.File(label="Upload T2 Modality (.nii)", type="file"),
        # Add more modalities if needed
    ],
    outputs=[
        gr.outputs.Textbox(label="Detection Result"),
        gr.outputs.Image(type="plot", label="Anomaly Detection Visualization")
    ],
    title="MRI Anomaly Detection",
    description="Upload MRI scans for different modalities to detect anomalies."
)

# Launch the interface
iface.launch(debug=True)


ModuleNotFoundError: No module named 'gradio'

In [6]:
!pip install gradio nibabel


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.7/56.7 MB 10.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 319.8/319.8 kB 20.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.7/94.7 kB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 447.5/447.5 kB 26.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.0/11.0 MB 94.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.3/73.3 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.7/63.7 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.2/130.2 kB 9.3 MB/s eta 0:00:00
  Attempting uninstall: markupsafe
    Found existing installation: MarkupSafe 3.0.2
    Uninstalling MarkupSafe-3.0.2:
      Successfully uninstalled MarkupSafe-3.0.2
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.24.7
    Uninstalling huggingface-hub-0.24.7:
      Successfully uninstalled huggingface-hub-0.24.7


In [ ]:
import gradio as gr
import nibabel as nib
import numpy as np
import torch
import cv2
import matplotlib.pyplot as plt

# Device setup
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Preprocessing and Averaging Function
def load_and_average_modalities(modalities, slice_axis=2, target_size=(256, 256)):
    processed_slices = []
    for file_path in modalities:
        # Load .nii file
        img = nib.load(file_path)
        data = img.get_fdata()

        # Extract the slice with the most non-zero pixels
        slices = [data.take(i, axis=slice_axis) for i in range(data.shape[slice_axis])]
        max_index = np.argmax([np.count_nonzero(slice_) for slice_ in slices])
        best_slice = slices[max_index]

        # Resize and normalize
        best_slice_resized = cv2.resize(best_slice, target_size)
        best_slice_normalized = best_slice_resized / np.max(best_slice_resized) if np.max(best_slice_resized) > 0 else best_slice_resized
        processed_slices.append(best_slice_normalized)

    # Average across modalities
    averaged_slice = np.mean(processed_slices, axis=0)
    return averaged_slice

# Main Function to Run the Anomaly Detection
def detect_anomaly(*modalities):
    # Step 1: Load and preprocess modalities
    averaged_slice = load_and_average_modalities(modalities)
    validation_tensor = torch.tensor(averaged_slice, dtype=torch.float32).unsqueeze(0).unsqueeze(0).to(device)

    # Step 2: Generate fake image and calculate reconstruction error
    noise = torch.randn((1, latent_dim), device=device)
    with torch.no_grad():
        generated_img = G(noise).squeeze().cpu().numpy()

    real_img_np = validation_tensor.squeeze().cpu().numpy()
    reconstruction_error = np.abs(real_img_np - generated_img)

    # Step 3: Thresholding for anomaly mask
    mean_error = np.mean(reconstruction_error)
    std_error = np.std(reconstruction_error)
    threshold = mean_error + 1.5 * std_error
    anomaly_mask = reconstruction_error > threshold
    anomaly_ratio = np.sum(anomaly_mask) / anomaly_mask.size
    result_text = "Tumor Found!" if anomaly_ratio > 0.04 else "No Tumor Found."

    # Step 4: Plot results
    fig, axes = plt.subplots(1, 4, figsize=(16, 4))
    axes[0].imshow(real_img_np, cmap='gray')
    axes[0].set_title("Original Averaged Image")
    axes[1].imshow(generated_img, cmap='gray')
    axes[1].set_title("Generated Image")
    axes[2].imshow(reconstruction_error, cmap='hot')
    axes[2].set_title("Reconstruction Error")
    axes[3].imshow(anomaly_mask, cmap='gray')
    axes[3].set_title("Anomaly Mask")

    plt.tight_layout()
    plt.close(fig)
    return result_text, fig

# Define Gradio Interface
iface = gr.Interface(
    fn=detect_anomaly,
    inputs=[
        gr.File(label="Upload T1CE Modality (.nii)"),
        gr.File(label="Upload T2 Modality (.nii)"),
        # Add more modalities if needed
    ],
    outputs=[
        gr.Textbox(label="Detection Result"),
        gr.Plot(label="Anomaly Detection Visualization")
    ],
    title="MRI Anomaly Detection",
    description="Upload MRI scans for different modalities to detect anomalies."
)

# Launch the interface
iface.launch(debug=True)


Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://ae643de7f04398a2e5.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
